In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, Activation, Dropout
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam, SGD


Using TensorFlow backend.


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/TrainData/TrainAnnotations.csv', dtype = 'str')
df['annotation'].value_counts()

0    488
1    329
4    197
3    131
2    130
Name: annotation, dtype: int64

In [ ]:
batch_size = 16

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 20,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.1,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(directory =  '/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/Train',
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical', 
                                                    target_size = (224, 224))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( directory = '/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/Validation',
                                                          batch_size  = batch_size,
                                                          class_mode  = 'categorical', 
                                                          target_size = (224, 224))

Found 911 images belonging to 5 classes.
Found 292 images belonging to 5 classes.


In [ ]:
# df = pd.read_csv('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/TrainAnnotationsWithAugmentation.csv', dtype = 'str')
# for i,row in df.iterrows():
#     if row["file_name"].split(".")[-1] != "jpg":
#         row["file_name"] += ".jpg"
# df.head()

In [ ]:
# df["annotation"].value_counts()

In [ ]:
# ##Flow from Data Frame and Augmented Data Folder

# datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.20)
# train_generator=datagen.flow_from_dataframe(
#             dataframe= df,
#             directory="/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/TrainDataWithAugmentation/",
#             x_col="file_name",
#             y_col="annotation",
#             subset="training",
#             batch_size=32,
#             seed=42,
#             shuffle=True,
#             class_mode="categorical",
#             target_size=(224,224))

# valid_generator=datagen.flow_from_dataframe(
#               dataframe= df,
#               directory="/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/TrainDataWithAugmentation/",
#               x_col="file_name",
#               y_col="annotation",
#               subset="validation",
#               batch_size=32,
#               seed=42,
#               shuffle=True,
#               class_mode="categorical",
#               target_size=(224,224))

In [ ]:
from keras.applications import vgg16
from keras.applications.densenet import DenseNet121
from keras.applications.inception_v3 import InceptionV3
base_model = vgg16.VGG16(input_shape=(224, 224, 3), weights='imagenet',include_top = False, pooling = 'avg')

len(base_model.layers)

58892288/58889256 [==============================] - 2s 0us/step


20

In [ ]:
for layer in base_model.layers[:15]:
  layer.trainable = False

for layer in base_model.layers[15:]:
  layer.trainable = True

In [ ]:
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x7f64fb5a9080>,input_1,False
1,<keras.layers.convolutional.Conv2D object at 0x7f64fb5a9630>,block1_conv1,False
2,<keras.layers.convolutional.Conv2D object at 0x7f64fb5a9a20>,block1_conv2,False
3,<keras.layers.pooling.MaxPooling2D object at 0x7f64fb5a9c18>,block1_pool,False
4,<keras.layers.convolutional.Conv2D object at 0x7f64fb5a9c50>,block2_conv1,False
5,<keras.layers.convolutional.Conv2D object at 0x7f64fb562c88>,block2_conv2,False
6,<keras.layers.pooling.MaxPooling2D object at 0x7f64fb577828>,block2_pool,False
7,<keras.layers.convolutional.Conv2D object at 0x7f64fb577668>,block3_conv1,False
8,<keras.layers.convolutional.Conv2D object at 0x7f64fb57e358>,block3_conv2,False
9,<keras.layers.convolutional.Conv2D object at 0x7f64fb57eef0>,block3_conv3,False


In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:

x = base_model.output
x = Dropout(0.5)(x)
x = Dense(512)(x)

x = Activation('relu')(x)
x = Dense(1024)(x)

x = Activation('relu')(x)
predictions = Dense(5, activation='softmax')(x)
vgg_model = Model(inputs=base_model.input, outputs=predictions)

vgg_model.compile(loss = "categorical_crossentropy", optimizer = 'SGD', metrics = ['accuracy'])

vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

def get_callbacks():

   path_checkpoint ='checkpoint_keras'  
   log_dir='logs'
   
   callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                         monitor='val_loss',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         period=1)
   
   callback_early_stopping = EarlyStopping(monitor='val_loss',
                                           patience=5,
                                           verbose=1)
   
   callback_tensorboard = TensorBoard(log_dir=log_dir,
                                      histogram_freq=0,
                                      write_graph=False)
   
   callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                          factor=0.1,
                                          min_lr=1e-4,
                                          patience=3,
                                          verbose=1)

   callbacks = [callback_checkpoint, callback_tensorboard, callback_reduce_lr]

   return callbacks

In [ ]:
epochs = 40

In [ ]:
vgg_model.fit_generator(train_generator, 
                    epochs = epochs, 
                    validation_data = validation_generator, callbacks = get_callbacks())



Epoch 1/40
57/57 [==============================] - 312s 5s/step - loss: 1.5360 - accuracy: 0.3172 - val_loss: 1.2999 - val_accuracy: 0.5788

Epoch 00001: val_loss improved from -inf to 1.29987, saving model to checkpoint_keras
Epoch 2/40
57/57 [==============================] - 19s 327ms/step - loss: 1.3615 - accuracy: 0.4138 - val_loss: 0.8352 - val_accuracy: 0.5548

Epoch 00002: val_loss did not improve from 1.29987
Epoch 3/40
57/57 [==============================] - 21s 367ms/step - loss: 1.2322 - accuracy: 0.4753 - val_loss: 0.9695 - val_accuracy: 0.6062

Epoch 00003: val_loss did not improve from 1.29987
Epoch 4/40
57/57 [==============================] - 21s 364ms/step - loss: 1.1725 - accuracy: 0.4874 - val_loss: 1.0598 - val_accuracy: 0.4589

Epoch 00004: val_loss did not improve from 1.29987
Epoch 5/40
57/57 [==============================] - 21s 371ms/step - loss: 1.1327 - accuracy: 0.5214 - val_loss: 0.5913 - val_accuracy: 0.6575

Epoch 00005: val_loss did not improve from 

In [ ]:
## vggbest AccuracyL 84 val_acc: 82

In [ ]:
history = vgg_model.history

import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/vgg_accuracy_curve.png')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/vgg_loss_curve.png')

NameError: ignored

In [ ]:
#Save the model
vgg_model.save('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/vggbest_model.h5')
with open('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/vggbest_model.json', 'w') as f:
    f.write(vgg_model.to_json())

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        directory = r"/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/TestData",
        target_size=(224, 224),
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
print(filenames)

Found 200 images belonging to 1 classes.
['0/020031.jpg', '0/020244.jpg', '0/020446.jpg', '0/020597.jpg', '0/020693.jpg', '0/020720.jpg', '0/020783.jpg', '0/020795.jpg', '0/020810.jpg', '0/020833.jpg', '0/020835.jpg', '0/020848.jpg', '0/020861.jpg', '0/020877.jpg', '0/020881.jpg', '0/020963.jpg', '0/020980.jpg', '0/021039.jpg', '0/021048.jpg', '0/021095.jpg', '0/021104.jpg', '0/021220.jpg', '0/021267.jpg', '0/021366.jpg', '0/021423.jpg', '0/021455.jpg', '0/021457.jpg', '0/021621.jpg', '0/021659.jpg', '0/021729.jpg', '0/021763.jpg', '0/021773.jpg', '0/021818.jpg', '0/021819.jpg', '0/022010.jpg', '0/022021.jpg', '0/022090.jpg', '0/022127.jpg', '0/022145.jpg', '0/022158.jpg', '0/022202.jpg', '0/022231.jpg', '0/022377.jpg', '0/022385.jpg', '0/022432.jpg', '0/022438.jpg', '0/022497.jpg', '0/022551.jpg', '0/022554.jpg', '0/022628.jpg', '0/022629.jpg', '0/022642.jpg', '0/022757.jpg', '0/022800.jpg', '0/022826.jpg', '0/023041.jpg', '0/023112.jpg', '0/023195.jpg', '0/023229.jpg', '0/023278.jpg'

In [ ]:
import numpy as np
# #load model

new_model = keras.models.load_model('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/vggbest_model.h5')
print("Model loaded successfully")

Model loaded successfully


In [ ]:
predictions = new_model.predict_generator(test_generator) 
print(predictions)    
predictions = np.argmax(predictions, axis=-1) #multiple categories
label = (test_generator.class_indices)
print(predictions)

[[1.32289872e-01 7.44562507e-01 1.19409360e-01 3.51464283e-03
  2.23555282e-04]
 [5.62089044e-05 1.74713507e-02 1.53311407e-02 4.56690267e-02
  9.21472251e-01]
 [1.10060446e-05 2.54654698e-03 3.48485890e-03 1.92639269e-02
  9.74693596e-01]
 [6.21736169e-01 3.32003653e-01 4.59530503e-02 3.03459674e-04
  3.60744275e-06]
 [7.60520762e-03 9.80801284e-01 1.13867577e-02 1.44227568e-04
  6.25913963e-05]
 [1.31713423e-05 8.70723801e-04 2.85965274e-03 3.08574326e-02
  9.65399027e-01]
 [1.31882336e-02 8.78503621e-01 1.04822785e-01 3.21480725e-03
  2.70642777e-04]
 [6.48657084e-01 2.83121228e-01 6.77568838e-02 4.59142990e-04
  5.66261224e-06]
 [2.17698514e-01 7.81128466e-01 1.11396064e-03 2.87117273e-05
  3.04029436e-05]
 [6.83143079e-01 3.16105306e-01 7.49248604e-04 2.30402475e-06
  1.19299870e-07]
 [1.60934348e-02 6.28125548e-01 2.91886479e-01 5.06028943e-02
  1.32916104e-02]
 [9.92413061e-06 4.55414224e-03 1.80627441e-03 6.43783528e-03
  9.87191796e-01]
 [3.19941289e-04 4.30238284e-02 1.111790

In [ ]:
test_set_dir = os.path.join('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/TestData/0')
test_names = os.listdir(test_set_dir )
print("Test data has: ", len(test_names) , " images." )
print(test_names[:10])
image_prediction_dict = {}
for x, y in zip(test_names,predictions):
  image_prediction_dict[x] = y
  
class0 = [k for k,v in image_prediction_dict.items() if v == 0]
class1 = [k for k,v in image_prediction_dict.items() if v == 1]
class2 = [k for k,v in image_prediction_dict.items() if v == 2]
class3 = [k for k,v in image_prediction_dict.items() if v == 3]
class4 = [k for k,v in image_prediction_dict.items() if v == 4]
print("\nClass 0 images: ", len(class0), "\nClass 1 images: ", len(class1), "\nClass 2 images: ", len(class2), "\nClass 3 images: " ,len(class3), "\nClass 4 images: ", len(class4))
print(image_prediction_dict)

Test data has:  200  images.
['027976.jpg', '027323.jpg', '024060.jpg', '025187.jpg', '024242.jpg', '020597.jpg', '029232.jpg', '029425.jpg', '022628.jpg', '029019.jpg']

Class 0 images:  37 
Class 1 images:  89 
Class 2 images:  17 
Class 3 images:  4 
Class 4 images:  53
{'027976.jpg': 1, '027323.jpg': 4, '024060.jpg': 4, '025187.jpg': 0, '024242.jpg': 1, '020597.jpg': 4, '029232.jpg': 1, '029425.jpg': 0, '022628.jpg': 1, '029019.jpg': 0, '024256.jpg': 1, '026347.jpg': 4, '022145.jpg': 4, '027886.jpg': 1, '025692.jpg': 0, '026463.jpg': 2, '023513.jpg': 1, '020031.jpg': 1, '029866.jpg': 1, '027898.jpg': 0, '029650.jpg': 1, '021819.jpg': 1, '027072.jpg': 3, '027495.jpg': 1, '023537.jpg': 2, '021455.jpg': 3, '023291.jpg': 4, '025668.jpg': 0, '023112.jpg': 1, '027132.jpg': 0, '029841.jpg': 4, '029484.jpg': 2, '021095.jpg': 4, '020810.jpg': 1, '024768.jpg': 1, '024739.jpg': 4, '025225.jpg': 4, '027228.jpg': 1, '025755.jpg': 1, '020980.jpg': 0, '023846.jpg': 1, '022757.jpg': 4, '029251.jpg

In [ ]:
from numpy import argmax
from keras.utils import to_categorical
from numpy import array
# define example
data = predictions
data = array(data)
print(data)
# one hot encode
encoded = to_categorical(data)
int_convert = []
for x in encoded:
  int_convert.append([int(k) for k in x])
print(int_convert)

import csv
csvfile = open('/content/drive/My Drive/Projects/Neural_Network_Competition_Project/Phase_2/predictvggbest.csv','w', newline='')
obj1 = csv.writer(csvfile)
obj1.writerows(int_convert)
csvfile.close()
print("File Created")

[1 4 4 0 1 4 1 0 1 0 1 4 4 1 0 2 1 1 1 0 1 1 3 1 2 3 4 0 1 0 4 2 4 1 1 4 4
 1 1 0 1 4 1 1 0 4 1 1 1 2 0 0 1 4 4 4 0 4 1 0 1 0 1 1 0 2 1 1 2 4 4 1 1 4
 0 3 4 1 1 1 0 1 4 2 1 4 1 0 1 4 4 1 1 1 1 1 0 4 0 1 3 4 4 2 0 1 4 0 1 1 1
 2 4 1 4 4 1 1 1 4 1 0 2 4 1 1 4 4 1 1 1 4 0 1 0 1 0 1 2 1 1 4 0 2 0 1 1 1
 4 4 0 1 2 1 4 1 0 2 1 0 1 1 0 1 2 0 4 4 4 0 4 4 0 1 4 1 1 4 4 4 1 0 4 1 1
 1 2 1 1 1 1 1 4 1 4 1 4 2 1 0]
[[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0

In [ ]:
filenames[6]

'0/020783.jpg'

In [ ]:
for i in range(len(filenames)):
  print(i + 1 , " : " , filenames[i])

1  :  0/020031.jpg
2  :  0/020244.jpg
3  :  0/020446.jpg
4  :  0/020597.jpg
5  :  0/020693.jpg
6  :  0/020720.jpg
7  :  0/020783.jpg
8  :  0/020795.jpg
9  :  0/020810.jpg
10  :  0/020833.jpg
11  :  0/020835.jpg
12  :  0/020848.jpg
13  :  0/020861.jpg
14  :  0/020877.jpg
15  :  0/020881.jpg
16  :  0/020963.jpg
17  :  0/020980.jpg
18  :  0/021039.jpg
19  :  0/021048.jpg
20  :  0/021095.jpg
21  :  0/021104.jpg
22  :  0/021220.jpg
23  :  0/021267.jpg
24  :  0/021366.jpg
25  :  0/021423.jpg
26  :  0/021455.jpg
27  :  0/021457.jpg
28  :  0/021621.jpg
29  :  0/021659.jpg
30  :  0/021729.jpg
31  :  0/021763.jpg
32  :  0/021773.jpg
33  :  0/021818.jpg
34  :  0/021819.jpg
35  :  0/022010.jpg
36  :  0/022021.jpg
37  :  0/022090.jpg
38  :  0/022127.jpg
39  :  0/022145.jpg
40  :  0/022158.jpg
41  :  0/022202.jpg
42  :  0/022231.jpg
43  :  0/022377.jpg
44  :  0/022385.jpg
45  :  0/022432.jpg
46  :  0/022438.jpg
47  :  0/022497.jpg
48  :  0/022551.jpg
49  :  0/022554.jpg
50  :  0/022628.jpg
51  :  0/

In [ ]:
predictions = vgg_model.predict_generator(validation_generator)
val_preds = np.argmax(predictions, axis=-1)
val_trues = validation_generator.classes
cm = metrics.confusion_matrix(val_trues, val_preds)

In [ ]:
val_trues

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4], dtype=int32)

In [ ]:
val_preds

array([4, 2, 0, 0, 0, 1, 0, 0, 2, 4, 0, 0, 4, 3, 0, 0, 0, 2, 1, 0, 0, 1,
       2, 0, 0, 0, 0, 1, 4, 1, 2, 2, 0, 0, 4, 2, 4, 1, 0, 0, 3, 0, 1, 2,
       3, 2, 4, 0, 2, 4, 3, 0, 4, 0, 0, 4, 0, 1, 2, 1, 0, 0, 1, 2, 4, 3,
       0, 0, 0, 4, 4, 1, 0, 2, 0, 2, 4, 1, 0, 3, 3, 1, 3, 3, 3, 0, 3, 0,
       4, 0, 4, 0, 3, 0, 1, 0, 0, 0, 4, 0, 2, 0, 4, 0, 4, 0, 4, 0, 0, 0,
       4, 4, 1, 1, 3, 1, 0, 0, 0, 3, 2, 0, 0, 0, 2, 3, 2, 0, 1, 0, 0, 3,
       2, 2, 1, 0, 1, 0, 1, 1, 0, 1, 2, 4, 0, 3, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 3, 2, 0, 0, 0, 2, 1, 1, 0, 3, 0, 1, 2, 2, 0, 2, 0, 2, 0, 0,
       4, 3, 2, 4, 0, 0, 0, 0, 3, 0, 4, 3, 0, 4, 3, 4, 0, 3, 0, 4, 0, 4,
       3, 0, 4, 1, 0, 4, 0, 4, 2, 0, 0, 2, 1, 3, 4, 1, 3, 2, 0, 3, 4, 0,
       2, 0, 3, 2, 2, 1, 0, 0, 2, 3, 4, 2, 1, 4, 1, 0, 0, 2, 1, 4, 0, 0,
       0, 4, 1, 1, 4, 0, 4, 4, 1, 1, 0, 2, 0, 2, 4, 4, 0, 3, 2, 3, 1, 3,
       2, 2, 1, 1, 3, 1, 2, 4, 1, 0, 4, 4, 0, 3, 3, 3, 3, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 1, 1])

In [ ]:
cm

array([[53, 17, 16, 14, 22],
       [31, 11, 12,  8,  4],
       [ 9,  1,  2,  3,  7],
       [10,  6,  7,  5,  5],
       [13, 11,  8,  8,  9]])

In [ ]:
precisions, recall, f1_score, _ = metrics.precision_recall_fscore_support(val_trues, val_preds)


In [ ]:
metrics.accuracy_score(val_trues, val_preds)

0.273972602739726

In [ ]:
precisions

array([0.45689655, 0.23913043, 0.04444444, 0.13157895, 0.19148936])

In [ ]:
recall

array([0.43442623, 0.16666667, 0.09090909, 0.15151515, 0.18367347])

In [ ]:
f1_score

array([0.44537815, 0.19642857, 0.05970149, 0.14084507, 0.1875    ])